# Board

In [118]:
class Node:
    def __init__(self, board, player):
        self.board = board[:]
        self.player = player
        self.children = []
        self.value = None

    def __repr__(self):
        b = self.board
        board_str = (f"\n {b[0]} | {b[1]} | {b[2]}"
                     f"\n---+---+---"
                     f"\n {b[3]} | {b[4]} | {b[5]}"
                     f"\n---+---+---"
                     f"\n {b[6]} | {b[7]} | {b[8]}")
        return f"Player: {self.player}, Value: {self.value}\n{board_str}\nChildren: {len(self.children)}"

def winner(board):
    win_positions = [
        (0,1,2), (3,4,5), (6,7,8),
        (0,3,6), (1,4,7), (2,5,8),
        (0,4,8), (2,4,6)
    ]
    for (a,b,c) in win_positions:
        if board[a] != " " and board[a] == board[b] == board[c]:
            return board[a]
    return None

def available_moves(board):
    return [i for i in range(9) if board[i] == " "]

def is_full(board):
    return " " not in board


# MinMax-Algo

In [119]:
def build_minimax_tree(node, checked_nodes=0):
    # Terminal condition
    win = winner(node.board)
    if win == "X":
        node.value = 1
        return node, checked_nodes
    elif win == "O":
        node.value = -1
        return node, checked_nodes
    elif is_full(node.board):
        node.value = 0
        return node, checked_nodes

    # Recursive expansion
    moves = available_moves(node.board)
    next_player = "O" if node.player == "X" else "X"

    for move in moves:
        new_board = node.board[:]
        new_board[move] = node.player
        child = Node(new_board, next_player)
        temp, checked_nodes = build_minimax_tree(child, checked_nodes)
        node.children.append(child)

    # Evaluate based on Minimax rule
    if node.player == "X":
        node.value = max(child.value for child in node.children)
    else:
        node.value = min(child.value for child in node.children)
    checked_nodes += len(node.children)
    return node, checked_nodes


In [120]:
initial_board = [" ", "O", "O",
                 " ", " ", " ",
                 " ", "O", "O"]
root = Node(initial_board, "X")
tree, checked_nodes = build_minimax_tree(root)
if tree.value == 1:
    print("X wins: ", tree.value)
elif tree.value == -1:
    print("O wins: ", tree.value)
else:
    print("Draw: ", tree.value)

print("Checked nodes", checked_nodes)

O wins:  -1
Checked nodes 61


# MinMax-Algo with pruning

In [121]:
def build_minimax_tree_pruning(node, alpha, beta, checked_nodes=0):
    # Terminal condition
    win = winner(node.board)
    if win == "X":
        node.value = 1
        return node, checked_nodes
    elif win == "O":
        node.value = -1
        return node, checked_nodes
    elif is_full(node.board):
        node.value = 0
        return node, checked_nodes

    moves = available_moves(node.board)
    next_player = "O" if node.player == "X" else "X"

    # Maximizing player (X)
    if node.player == "X":
        node.value = float('-inf')
        for move in moves:
            new_board = node.board[:]
            new_board[move] = node.player
            child = Node(new_board, next_player)

            child, checked_nodes = build_minimax_tree_pruning(child, alpha, beta, checked_nodes)
            node.children.append(child)

            node.value = max(node.value, child.value)
            alpha = max(alpha, node.value)

            if beta <= alpha:
                break

            checked_nodes += 1

    # Minimizing player (O)
    else:
        node.value = float('inf')
        for move in moves:
            new_board = node.board[:]
            new_board[move] = node.player
            child = Node(new_board, next_player)

            child, checked_nodes = build_minimax_tree_pruning(child, alpha, beta, checked_nodes)
            node.children.append(child)

            node.value = min(node.value, child.value)
            beta = min(beta, node.value)

            if beta <= alpha:
                break

            checked_nodes += 1

    return node, checked_nodes

In [122]:
initial_board = [" ", "O", "O",
                 " ", " ", " ",
                 " ", "O", "O"]
root = Node(initial_board, "X")
tree, checked_nodes = build_minimax_tree_pruning(root, alpha=-2, beta=2)
if tree.value == 1:
    print("X wins: ", tree.value)
elif tree.value == -1:
    print("O wins: ", tree.value)
else:
    print("Draw: ", tree.value)

print("Checked nodes", checked_nodes)

O wins:  -1
Checked nodes 14
